# Selección de características en modelos predictivos

Este proyecto es para IA de Moisés Calzado Cobo y Antonio Germán Márquez Trujillo

Estos archivos contienen los algoritmos implementados, pero no los importamos ya que en esta libreta los implementamos explicando su funcionamiento


In [5]:
import pandas as pandas
import time
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing as mp
from sklearn.model_selection import cross_val_score
from sklearn import tree
from funcy import join
from sklearn import preprocessing
pandas.set_option('max_colwidth', 800)

In [2]:
datos = pandas.read_csv("datos/titanic.csv")
#datos = pandas.read_csv("datos/BreastCancerDataset.csv")
datos.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Initial,Age_band,Family_Size,Alone,Fare_cat,Deck,Title,Is_Married,Survived
0,3,0,22.0,1,0,7.2500,0,0,1,1,0,0,7,11,0,0
1,1,1,38.0,1,0,71.2833,1,1,2,1,0,3,2,12,1,1
2,3,1,26.0,0,0,7.9250,0,2,1,0,1,1,7,8,0,1
3,1,1,35.0,1,0,53.1000,0,1,2,1,0,3,2,12,1,1
4,3,0,35.0,0,0,8.0500,0,0,2,0,1,1,7,11,0,0


In [3]:
def evaluar_soluciones(datos, solucion_actual, nueva_variable, objetivo, n_exp, cv):
    variables = solucion_actual[:]
    variables.append(nueva_variable)
    data_frame = pandas.DataFrame(data=datos)
    X = data_frame[variables]
    y = data_frame[objetivo]
    clf = tree.DecisionTreeClassifier()
    
    scores = cross_val_score(clf, X, y, cv=cv, scoring="balanced_accuracy")
    
    for i in range(n_exp-1):
        new_scores = cross_val_score(clf, X, y, cv=cv, scoring="balanced_accuracy") 
        scores = scores + new_scores
    
    scores = scores/n_exp
    clave = ', '.join(variables)
    
    diccionario_resultado = {}
    diccionario_resultado[clave] = np.mean(scores)
    return diccionario_resultado

Algoritmo SFS paralelizado:

In [8]:
def SFS(datos, respuesta, d = 0):
    start = time.time()

    diccionario_resultado = {}
    columnas = list(datos.columns)
    columnas.remove(respuesta)
    solucion_actual = []

    k = 0
    d = d if d else len(columnas)
    
    while(k<d):
        pool = mp.Pool(mp.cpu_count())
        new_resultados = pool.starmap(evaluar_soluciones, [(datos, solucion_actual, nuevaVariable, respuesta, 15, 10) for nuevaVariable in columnas])
        pool.close()
        resultado = join(new_resultados)
        
        variable_escogida = max(resultado, key=resultado.get)
        variables_nuevas = variable_escogida.split(", ")
        solucion_actual.append(variables_nuevas[len(variables_nuevas)-1])
        columnas.remove(variables_nuevas[len(variables_nuevas)-1])

        k = k+1
        
        diccionario_resultado[variable_escogida] = resultado[variable_escogida]
    
    done = time.time()
    elapsed = done - start
    print(elapsed)
    return diccionario_resultado

In [9]:
resultado_SFS = SFS(datos, "Survived", 3)
print(resultado_SFS)

13.290151834487915
{'Initial': 0.7833541295306, 'Initial, SibSp': 0.8052182893359363, 'Initial, SibSp, Deck': 0.8077201991907874}


Algoritmo SFFS paralelizado:

In [ ]:
def SFFS(datos, respuesta):
    diccionario_resultado = {}
    soluciones_actual = []
    añadidos = []
    eliminados = []
    columnas = list(datos.columns)
    k = 0
    
    #Compruebo que la variable a predecir no esté en mi conjunto de variables a evaluar
    if respuesta in columnas:
        columnas.remove(respuesta)
    
    while(k<10):
        resultado = []
        score_resultado = 0
        score_resultado_eliminado = 0
        resultado_eliminado = []
        eliminado = ''

        for i in range(len(columnas)):
            
            #Compruebo que la nueva variable a evaluar no haya sido ya evaluada o este en añadidos
            if columnas[i] not in soluciones_actual and columnas[i] not in añadidos:
                solucionTemporal = list(soluciones_actual)
                solucionTemporal.append(columnas[i])
                new_resultado = evaluar_soluciones(datos, solucionTemporal, respuesta, 15, 10)
                
                #Si el resultado es favorable, actualizo el resultado final
                if new_resultado > score_resultado:
                    resultado = solucionTemporal
                    score_resultado = new_resultado
                    
        if len(resultado)>0:
            soluciones_actual.append(resultado[len(resultado)-1])
            añadidos.append(resultado[len(resultado)-1])
        
        score_resultado_eliminado = score_resultado
        
        if len(soluciones_actual)>1:
            for i in range(len(soluciones_actual)):

                #Compruebo que la variable a evaluar no este en eliminados
                if soluciones_actual[i] not in eliminados:
                    solucionTemporal = list(soluciones_actual)
                    solucionTemporal.remove(soluciones_actual[i])
                    new_resultado = evaluar_soluciones(datos, solucionTemporal, respuesta, 15, 10)

                    #Si el resultado es favorable, actualizo el resultado para eliminar la variable actual que
                    #ha sido quitada de la solución actual
                    if new_resultado > score_resultado_eliminado:
                        resultado_eliminado = solucionTemporal
                        score_resultado_eliminado = new_resultado
                        eliminado = soluciones_actual[i]
                    
            if score_resultado<score_resultado_eliminado:
                soluciones_actual = resultado_eliminado
                eliminados.append(eliminado)
                k=0
        
        k = k+1
        
        if len(añadidos)<len(columnas):
            clave = ', '.join(soluciones_actual)
            diccionario_resultado[clave] = score_resultado_eliminado
                    
    return diccionario_resultado

In [ ]:
resultado_SFFS = SFFS(datos, "Survived")
print(resultado_SFFS)